## Logistic regression

This notebook compares various logistic regression implementations. The dataset used is the [New South Wales electricity price dataset](https://online-ml.github.io/generated/river.datasets.fetch_electricity.html#river.datasets.fetch_electricity).

In [1]:
%load_ext watermark
%watermark --python --machine --packages river,keras,sklearn,tensorflow,torch --datename

Using TensorFlow backend.


Fri Jul 10 2020 

CPython 3.7.7
IPython 7.16.1

river 0.6.1
keras 2.3.1
sklearn 0.23.1
tensorflow 2.0.0
torch 1.5.1

compiler   : Clang 4.0.1 (tags/RELEASE_401/final)
system     : Darwin
release    : 19.5.0
machine    : x86_64
processor  : i386
CPU cores  : 12
interpreter: 64bit


In [2]:
from river import datasets
from river import linear_model
from river import metrics
from river import optim
from river import preprocessing
from keras import layers
from keras import models
from keras import optimizers
from sklearn import linear_model as sk_linear_model
import torch

%run utils.py
%run wrappers.py

In [ ]:
n_features = 8
lr = 0.005

# PyTorch
class PyTorchNet(torch.nn.Module):
    
    def __init__(self, n_features):
        super().__init__()
        self.linear = torch.nn.Linear(n_features, 1)
        self.sigmoid = torch.nn.Sigmoid()
        torch.nn.init.constant_(self.linear.weight, 0)
        torch.nn.init.constant_(self.linear.bias, 0)
        
    def forward(self, x):
        return self.sigmoid(self.linear(x))
    
torch_model = PyTorchNet(n_features=n_features)

# Keras
inputs = layers.Input(shape=(n_features,))
predictions = layers.Dense(1, activation='sigmoid', kernel_initializer='zeros', bias_initializer='zeros')(inputs)
keras_model = models.Model(inputs=inputs, outputs=predictions)
keras_model.compile(optimizer=optimizers.SGD(lr=lr), loss='binary_crossentropy')


def add_hour(x):
    x['hour'] = x['moment'].hour
    return x

results = benchmark(
    get_X_y=datasets.fetch_electricity,
    n=45312,
    get_pp=preprocessing.StandardScaler,
    models=[
        ('river', '', linear_model.LogisticRegression(
            optimizer=optim.SGD(lr),
            l2=0.,
            intercept_lr=lr
        )),

        ('scikit-learn', '', ScikitLearnClassifier(
            model=sk_linear_model.SGDClassifier(
                loss='log',
                learning_rate='constant',
                eta0=lr,
                penalty='none'
            ),
            classes=[False, True]
        )),
        
        ('PyTorch (CPU)', '', PyTorchBinaryClassifier(
            network=torch_model,
            loss_fn=torch.nn.BCELoss(),
            optimizer=torch.optim.SGD(torch_model.parameters(), lr=lr)
        )),
        
        ('Keras on Tensorflow (CPU)', '', KerasBinaryClassifier(
            model=keras_model
        )),
        
        ('VowpalWabbit', '', VW2CremeClassifier(
            sgd=True,
            learning_rate=lr,
            loss_function='logistic',
            link='logistic',
            adaptive=False,
            normalized=False,
            invariant=False,
            l2=0.,
            l1=0.,
            power_t=0
        )),
        
    ],
    get_metric=metrics.LogLoss
)

In [4]:
results

,Library,Model,LogLoss,Fit time,Average fit time,Predict time,Average predict time
0,river,LogisticRegression,0.413533,"812ms, 676μs, 174ns","17μs, 935ns","245ms, 461μs, 635ns","5μs, 417ns"
1,scikit-learn,SGDClassifier,0.413533,"18s, 902ms, 205μs, 239ns","417μs, 157ns","6s, 652ms, 347μs, 9ns","146μs, 812ns"
2,PyTorch (CPU),Linear,0.418901,"30s, 274ms, 892μs, 26ns","668μs, 143ns","10s, 904ms, 302μs, 161ns","240μs, 649ns"
3,Keras on Tensorflow (CPU),Dense,0.418901,"1m, 12s, 988ms, 526μs, 625ns","1ms, 610μs, 799ns","38s, 130ms, 826μs, 237ns","841μs, 517ns"


In [6]:
print(results.to_html())

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Library</th>
      <th>Model</th>
      <th>LogLoss</th>
      <th>Fit time</th>
      <th>Average fit time</th>
      <th>Predict time</th>
      <th>Average predict time</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>river</td>
      <td>LogisticRegression</td>
      <td>0.413533</td>
      <td>812ms, 676μs, 174ns</td>
      <td>17μs, 935ns</td>
      <td>245ms, 461μs, 635ns</td>
      <td>5μs, 417ns</td>
    </tr>
    <tr>
      <th>1</th>
      <td>scikit-learn</td>
      <td>SGDClassifier</td>
      <td>0.413533</td>
      <td>18s, 902ms, 205μs, 239ns</td>
      <td>417μs, 157ns</td>
      <td>6s, 652ms, 347μs, 9ns</td>
      <td>146μs, 812ns</td>
    </tr>
    <tr>
      <th>2</th>
      <td>PyTorch (CPU)</td>
      <td>Linear</td>
      <td>0.418901</td>
      <td>30s, 274ms, 892μs, 26ns</td>
      <td>668μs, 143ns</td>
      <td>10s, 904ms, 302μs